In [6]:
import numpy as np
import math
import random
import os
import os.path
import torch
import sys
import copy
import pickle
import importlib
import torch.nn as nn
import time
import functorch
from numpy.random import default_rng
from tqdm.notebook import tqdm

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import matplotlib.pyplot as plt
from chamferdist import ChamferDistance
from pathlib import Path
import open3d as o3d

from src.dataset import *
from src.elements import *


In [2]:
random.seed = 42
rng = default_rng()

In [3]:
path = Path('output/')
savepath = 'meta/models/'
cuda = torch.device('cuda')


In [8]:
train_transforms = transforms.Compose([
                    Normalize(),
#                    RandomNoise(),
                    ToTensor()
                    ])

In [9]:
# load data and model
BASE_DIR = os.path.dirname(os.path.abspath('industrial-facility-relationships/'))
BASE_DIR = os.path.join(BASE_DIR, 'pointnet2')
ROOT_DIR = BASE_DIR
sys.path.append(os.path.join(ROOT_DIR, 'models'))


path = Path('output/')    
ext = ".pcd"

cat= 'elbow'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
use_normals = False

test_ds = PointCloudData(path, valid=True, folder='test', category=cat, transform=train_transforms)
targets = test_ds.targets

testDataLoader = torch.utils.data.DataLoader(dataset=test_ds, batch_size=128)
test_criterion = nn.MSELoss()

model_name = "pointnet2_cls_ssg"
model_path = Path("pointnet2/log/classification/pointnet2_meta_ssg/")
model = importlib.import_module(model_name)


predictor = model.get_model(targets, normal_channel=use_normals)
if device != "cpu":
    predictor = predictor.cuda()

# #checkpoint = torch.load(model_path/'checkpoints/best_model.pth')
# checkpoint = torch.load(model_path/'checkpoints/models/best_model_f_chamfer_0005.pth')
# #checkpoint = torch.load(model_path/'checkpoints/models/best_model_t_chamfer_00005_bp.pth')
# predictor.load_state_dict(checkpoint['model_state_dict'])


RuntimeError: Error(s) in loading state_dict for get_model:
	size mismatch for fc3.weight: copying a param with shape torch.Size([13, 256]) from checkpoint, the shape in current model is torch.Size([14, 256]).
	size mismatch for fc3.bias: copying a param with shape torch.Size([13]) from checkpoint, the shape in current model is torch.Size([14]).